In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from archaeo_super_prompt.dataset import MagohDataset

In [3]:
from archaeo_super_prompt.modeling.pdf_to_text import VLLM_Preprocessing

In [4]:
scanner = VLLM_Preprocessing(
    vlm_provider="vllm",
    vlm_model_id="ibm-granite/granite-vision-3.3-2b",
    incipit_only=True,
    prompt="OCR this part of Italian document for markdown-based processing.",
    embedding_model_hf_id="nomic-ai/nomic-embed-text-v1.5",
)

In [5]:
_digitally_born_documents = [
    # very good
    33799, 34439, 38005, 36837, 36937, 37614, 37026, 37971, 36846, 36304, 34423, 36052,
    37043, 36554, 989, 37007, 30897, 36351, 36308, 38013, 36011, 33828, 1221,
    38039, 35429, 37065, 37116, 34452, 33441, 33062, 34939, 35918, 33689, 34508, 31035,
    38220, 38092, 36979, 36854, 36207, 34915, 35688, 36359,
    # not that good
    31164, 32600, 33760, 32714, 31208, 30712,
    ]
ok_scanned_pdfs = {
    32666, 31298, 33548, 35189, 35399, 30925, 37040, 37379, 33589, 34769,
    33858, 34329, 5193, 37706, 30647, 37702, 33540, 36042, 33357, 34959,
    30646, 33547, 32581, 30878, 37302, 33560, 35881, 31031, 37381, 242, 34869,
    33841, 36465, 33499, 36095, 36068, 33594, 33904, 33644, 33553, 35052,
    33630, 34426, 31090, 30716, 31059, 35849, 33813, 34666, 36119, 830,
    36187, 31977, 34787, 33749, 35447, 33555, 33846, 34093, 33508,  # 33710
}
dirty_pdfs = {
    36648, 32433, 35131, 33383, 30657, 31312, 30399, 33331, 31234, 30548,
    34685, 34237, 35114, 30821, 33708, 33668, 34932, 30697, 38241, 33443,
    37305, 33535, 31815, 35203, 33576, 32053, 33761, 37910, 35983, 31314,
    37400, 36457, 33582, 31903, 32494, 33184, 36070, 31804, 30861
}

selected_ids = set(_digitally_born_documents).union(ok_scanned_pdfs, dirty_pdfs)
ds = MagohDataset(selected_ids)
inputs = ds.files

In [7]:
inputs.iloc[:83]

,id,filepath
0,32053,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...
1,32053,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...
2,36187,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...
3,36308,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...
4,36042,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...
...,...,...
78,32600,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...
79,34915,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...
80,36070,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...
81,30861,/raid/ggattiglia/magoh_ai/archaeo-extraction-p...


In [10]:
output = scanner.transform(inputs.iloc[:80])

Chunking read text:   0%|                                                                     | 0/80 [00:00<?, ?chunked files/s]

Doc n°32053's scanned proportion:   0%|                                                           | 0/2 [00:00<?, ?page batch/s]

Doc n°32053's scanned proportion:  50%|█████████████████████████▌                         | 1/2 [00:00<00:00,  2.49page batch/s]


(32053, 3-4) rescanned pages:   0%|                                                                     | 0/2 [00:00<?, ?page/s]


(32053, 3-4) rescanned pages: 100%|█████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.40page/s]


Doc n°32053's scanned proportion: 100%|███████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.62page batch/s]

vision-llm-scanned files:   1%|▊                                                               | 1/80 [00:01<01:38,  1.24s/file]

Doc n°32053's scanned proportion:   0%|                                                

In [17]:
len(output["id"].unique())

71

In [16]:
from archaeo_super_prompt.utils.cache import get_cache_dir_for

output.to_csv(get_cache_dir_for("interim", "miscel") / "scans.csv")